In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3796782751862527033
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2966780673176528267
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13814361890876728115
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11150664704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15139474462186551803
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [3]:
!pip install bert-tensorflow


     |████████████████████████████████| 71kB 4.0MB/s 


In [4]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 6.5MB/s 


In [5]:
import pandas as pd
import numpy as np
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [6]:
real = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_real_tweets.csv', header = None)
fake = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_fake_tweets_unconditional.csv', header = None)

In [7]:
real.columns = ['tweet']
real['index'] = real.index
fake.columns = ['tweet']
fake['index'] = fake.index
cols = real.columns.tolist()
cols = cols[-1:] + cols[:-1]
real = real[cols]
cols = fake.columns.tolist()
cols = cols[-1:] + cols[:-1]
fake = fake[cols]

In [8]:
real['label'] = 1
fake['label'] = 0
real.head()

,index,tweet,label
0,0,#Taiwan isn't a member of the #WHO. Taiwan tr...,1
1,1,From the DJs to the CJs.. Look whose in the PJ...,1
2,2,The @ClinicalBioinfo led by @xdopazo releases ...,1
3,3,IMF expects #Greece to be the worst hit econom...,1
4,4,More on #COVID19 - China witnessed a slight in...,1


In [9]:
fake.head()

,index,tweet,label
0,0,@chrislhayes I think it's important to note th...,0
1,1,"The problem is, it's the Chinese that are bein...",0
2,2,Do you know of any people who are looking for ...,0
3,3,Just as the COVID-19 pandemic has shown that t...,0
4,4,Covid-19's impact on the economy: The RBC Stat...,0


In [10]:
combined = pd.concat([fake, real])
features = combined['tweet']
labels = combined['label']
combined = combined.sample(frac = 1).reset_index(drop = True)
combined['index'] = combined.index
combined.tail()

,index,tweet,label
19992,19992,If they give up they can lose their job,0
19993,19993,Quarantine sucks but I’ve legit seen some of t...,1
19994,19994,"In this #Covid_19 crisis, we will reach a peak...",0
19995,19995,This is horrifying. The @UN has issued an ur...,0
19996,19996,@mojoobii I was just waiting for the time to r...,0


In [11]:
from collections import Counter
print(Counter(combined['label'].values))

Counter({1: 10000, 0: 9997})


In [12]:
combined.dropna(inplace=True)

In [13]:
# x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)

In [14]:
train_data = combined.head(40)
test_data = combined.tail(8)

In [15]:
len(combined)

19996

In [16]:
train_data = [{'tweet': text, 'label': type_data } for text in list(train_data['tweet']) for type_data in list(train_data['label'])]

In [17]:
test_data = [{'tweet': text, 'label': type_data } for text in list(test_data['tweet']) for type_data in list(test_data['label'])]

In [18]:
train_texts, train_labels = list(zip(*map(lambda d: (d['tweet'], d['label']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['tweet'], d['label']), test_data)))

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

100%|██████████| 231508/231508 [00:00<00:00, 617476.76B/s]


In [20]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [21]:
train_y = np.array(train_labels)
test_y = np.array(test_labels)

In [22]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [23]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [24]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [25]:
BATCH_SIZE = 1

In [26]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [27]:
BATCH_SIZE = 1
EPOCHS = 1
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

100%|██████████| 407873900/407873900 [00:15<00:00, 26336721.61B/s]


Epoch:  1
0/1600.0 loss: 0.9541463851928711 
Epoch:  1
1/1600.0 loss: 1.0451714396476746 
Epoch:  1
2/1600.0 loss: 0.929798404375712 
Epoch:  1
3/1600.0 loss: 0.8928224146366119 
Epoch:  1
4/1600.0 loss: 0.871074891090393 
Epoch:  1
5/1600.0 loss: 0.8680546879768372 
Epoch:  1
6/1600.0 loss: 0.8259249414716449 
Epoch:  1
7/1600.0 loss: 0.7887037917971611 
Epoch:  1
8/1600.0 loss: 0.7838665246963501 
Epoch:  1
9/1600.0 loss: 0.7549756765365601 
Epoch:  1
10/1600.0 loss: 0.7618174336173318 
Epoch:  1
11/1600.0 loss: 0.7643238057692846 
Epoch:  1
12/1600.0 loss: 0.7586195973249582 
Epoch:  1
13/1600.0 loss: 0.7611037918499538 
Epoch:  1
14/1600.0 loss: 0.7558009386062622 
Epoch:  1
15/1600.0 loss: 0.7598051242530346 
Epoch:  1
16/1600.0 loss: 0.7549697721705717 
Epoch:  1
17/1600.0 loss: 0.7489722404215071 
Epoch:  1
18/1600.0 loss: 0.7512599072958294 
Epoch:  1
19/1600.0 loss: 0.7542297452688217 
Epoch:  1
20/1600.0 loss: 0.7553310110455468 
Epoch:  1
21/1600.0 loss: 0.7488402101126584 


In [28]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

           0       0.75      0.12      0.21        48
           1       0.25      0.88      0.39        16

    accuracy                           0.31        64
   macro avg       0.50      0.50      0.30        64
weighted avg       0.62      0.31      0.26        64

